In [1]:

#Working Copy  2022-02-25 0900AM
#pip install BeautifulSoup as b  & #pip install requests

from bs4 import BeautifulSoup as b
import numpy as np
import requests as r
import pandas as p
import math as m
import csv as cv
from datetime import date, timedelta, datetime

# getting search days from user
def getDates(txt):
    msg1 = 'Enter the search ' + txt + ' in YYYY-MM-DD format'
    startDate = input(msg1)
    gDate = 0 

    while gDate != 1:
        try:
            Syear, Smonth, Sday = map(int, startDate.split('-'))
            date1 = date(Syear, Smonth, Sday)
        except: 
            startDate = input(msg1)
            continue
        gDate = 1 
    return date1
date1 = getDates('startDate')
date2 = getDates('endDate')

#Get new search string
base = date1
stringA = "https://www.fpds.gov/ezsearch/fpdsportal?q=+SIGNED_DATE%3A%5B"
base = date1
endt = date2
stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=30&start="
stringC = '30'
myURLsearch = (stringA + stringB + stringC)
myURLsearch
downloadURL = r.get(myURLsearch,headers={})
soup = b(downloadURL.text,"html.parser")
#CSS
fulltable_select  = soup.select('span.results_heading')
myLen = m.ceil(int(fulltable_select[1].find_all('b')[2].text))
print(myLen)

runInt = 1
wID = 0
while wID < myLen + 60:
    endt = base + timedelta(days=1)
    stringB = str(base.year)+"%2F"+base.strftime("%m") + "%2F" + base.strftime("%d") + "%2C"+str(endt.year)+"%2F" + endt.strftime("%m") + "%2F" +endt.strftime("%d") + "%5D&s=FPDS.GOV&templateName=1.5.2&indexName=awardfull&x=24&y=7&start="
    stringC = int(stringC) + 30
    stringC = str(stringC)
    myURLsearch = (stringA + stringB + stringC)
    downloadURL = r.get(myURLsearch,headers={})
    soup = b(downloadURL.text,"html.parser")
    fulltable_select  = soup.select('span.results_text')
    datalist = []
    slop =[]
    for d in fulltable_select: 
        try: 
            datalist.append(d.a.contents)
        except:
            try:
                datalist.append(d.span.contents)
            except:
                #formerly known as slop
                datalist.append(str(d.contents).replace("\\t","").replace("\\n","").replace("[","").replace("]","").replace("\'",""))
            continue
    newlist = ["".join(d) if type(d) is list else d for d in datalist ]
    dlarr = np.array(newlist).reshape(30,19)
    #[",".join(d) if type(d) is list else   arr in dlarr]
    df = p.DataFrame(data= dlarr
    , columns=['AwardID','AwardType','LegalBusinessName','ContractingAgency','DateSigned','ActionObligation','ReferencedIDV','ContractingOffice','NAICS_Code','PSC_Code','EntityCity','UniqueEntityID_DUNS', 'EntityState','UniqueEntityID_SAM','Zip','UltimateParentUniqueEntityID_DUNS','UltimateParentLegalBusinessName','UltimateParentUniqueEntityID_SAM','CAGE_Code']
    )
    r2 = datetime.today().strftime("%Y%m%d%H%M%S")
            
    # Create run interval interger
    if wID < 1:
        fileName = 'fpds_ng_' + r2 + '.csv'
        df.to_csv(fileName,index = False)
        wID = wID + 30
    else:
        with open(fileName, "a", newline="") as file:
            writer = cv.writer(file,delimiter= ",",)
            writer.writerows(dlarr)
            wID = wID + 30
            print(wID)
#base = base + timedelta(days=1);

5733
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280
2310
2340
2370
2400
2430
2460
2490
2520
2550
2580
2610
2640
2670
2700
2730
2760
2790
2820
2850
2880
2910
2940
2970
3000
3030
3060
3090
3120
3150
3180
3210
3240
3270
3300
3330
3360
3390
3420
3450
3480
3510
3540
3570
3600
3630
3660
3690
3720
3750
3780
3810
3840
3870
3900
3930
3960
3990
4020
4050
4080
4110
4140
4170
4200
4230
4260
4290
4320
4350
4380
4410
4440
4470
4500
4530
4560
4590
4620
4650
4680
4710
4740
4770
4800
4830
4860
4890
4920
4950
4980
5010
5040
5070
5100
5130
5160
5190
5220
5250
5280
5310
5340
5370
5400
5430
5460
5490
5520
5550
5580
5610
5640
5670
5700
5730
5760


In [ ]:
%whos